In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
from core.utils.ql_loader import ql
from datetime import datetime, timedelta
from typing import List

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime, datetime_to_ql_date

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

In [3]:
curve = "USD-SOFR-1D"
ff = FixingsFetcher(fred_api_key=config["FRED_API_KEY"]) 
fixings = ff.get_fixings(curve)

In [6]:
usd_ois = IRSwaps(
    curve=curve,
    data_source="CME",
    fixings=fixings,
    ql_interpolation_algo="log_linear",
    error_verbose=True,
    max_njobs=-1,
)

In [7]:
curves = usd_ois.fetch_ql_irswap_curves(start_date=datetime(2025, 1, 1), end_date=datetime(2025, 5, 16))

In [11]:
ts_df = usd_ois.irswaps_timeseries_builder(
    start_date=datetime(2025, 1, 1),
    end_date=datetime(2025, 5, 10),
    queries=[
        IRSwapQuery(tenor="10Yx10Y", structure=IRSwapStructure.OUTRIGHT),
        IRSwapQuery(tenor="1Yx10Y", structure=IRSwapStructure.OUTRIGHT),
        IRSwapQuery(tenor="05Yx10Y", structure=IRSwapStructure.OUTRIGHT),
    ],
    n_jobs=-1,
)

PRICING IRSWAPS...: 0it [00:00, ?it/s]


In [12]:
ts_df

,USD-SOFR-1D 1Yx10Y OUTRIGHT RATE,USD-SOFR-1D 05Yx10Y OUTRIGHT RATE,USD-SOFR-1D 10Yx10Y OUTRIGHT RATE
Date,,,
2025-01-02 17:00:00-05:00,4.083285,4.186993,4.166629
2025-01-03 17:00:00-05:00,4.102139,4.192378,4.175864
2025-01-06 17:00:00-05:00,4.146289,4.239025,4.218981
2025-01-07 17:00:00-05:00,4.223941,4.328716,4.304253
2025-01-08 17:00:00-05:00,4.236097,4.353597,4.332603
...,...,...,...
2025-05-05 17:00:00-04:00,3.840814,4.237892,4.346109
2025-05-06 17:00:00-04:00,3.801746,4.214098,4.339302
2025-05-07 17:00:00-04:00,3.764838,4.170393,4.304619
